In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - \ done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425344 sha256=b6677b795b96990b0ca32daf68327213e7b4e2c2934f23497d2a55d2143a8040
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [2]:
import pyspark.pandas as pyd
import pandas as pd

/opt/conda/lib/python3.10/site-packages/pyspark/pandas/__init__.py:50: UserWarning: 'PYARROW_IGNORE_TIMEZONE' environment variable was not set. It is required to set this environment variable to '1' in both driver and executor sides if you use pyarrow>=2.0.0. pandas-on-Spark will set it for you but it does not work if there is a Spark context already launched.
  warnings.warn(


In [3]:
df = pyd.from_pandas(pd.read_excel("/kaggle/input/trending-fifa-players-dataset/trending_football_players.xlsx"))

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/12/24 20:05:13 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
df.head(5)

,Player_name,Images,Age,National_team,Positions,Overall,Potential_overall,Current_club,Current_contract,Value,Wage,Total_stats
0,T. Almada,https://cdn.sofifa.net/players/245/371/24_60.png,22,Argentina,"['CAM', 'CM', 'CF']",79,87,Atlanta United,2022 ~ 2025,€39.5M,€10K,2050
1,L. Palma,https://cdn.sofifa.net/player_0.svg,23,Honduras,['LW'],69,75,Celtic,2023 ~ 2028,€2.2M,€22K,1794
2,R. Lavia,https://cdn.sofifa.net/players/263/620/24_60.png,19,Belgium,['CDM'],73,86,Chelsea,2023 ~ 2030,€7M,€32K,1829
3,W. Zaïre-Emery,https://cdn.sofifa.net/players/270/673/24_60.png,17,France,"['CM', 'CDM']",77,89,Paris Saint Germain,2022 ~ 2025,€24M,€9K,2080
4,Gabri Veiga,https://cdn.sofifa.net/players/258/729/24_60.png,21,Spain,"['CM', 'CAM']",78,89,Al Ahli Jeddah,2023 ~ 2026,€31.5M,€28K,1944


In [5]:
df.dtypes

Player_name          object
Images               object
Age                   int64
National_team        object
Positions            object
Overall               int64
Potential_overall     int64
Current_club         object
Current_contract     object
Value                object
Wage                 object
Total_stats           int64
dtype: object

In [6]:
df.info()

<class 'pyspark.pandas.frame.DataFrame'>
Int64Index: 3000 entries, 0 to 2999
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Player_name        3000 non-null   object
 1   Images             3000 non-null   object
 2   Age                3000 non-null   int64 
 3   National_team      3000 non-null   object
 4   Positions          3000 non-null   object
 5   Overall            3000 non-null   int64 
 6   Potential_overall  3000 non-null   int64 
 7   Current_club       3000 non-null   object
 8   Current_contract   3000 non-null   object
 9   Value              3000 non-null   object
 10  Wage               3000 non-null   object
 11  Total_stats        3000 non-null   int64 
dtypes: int64(4), object(8)

In [7]:
df.columns

Index(['Player_name', 'Images', 'Age', 'National_team', 'Positions', 'Overall',
       'Potential_overall', 'Current_club', 'Current_contract', 'Value',
       'Wage', 'Total_stats'],
      dtype='object')

In [8]:
df.shape

(3000, 12)

In [9]:
df.loc[df['Player_name']=='T. Almada']

,Player_name,Images,Age,National_team,Positions,Overall,Potential_overall,Current_club,Current_contract,Value,Wage,Total_stats
0,T. Almada,https://cdn.sofifa.net/players/245/371/24_60.png,22,Argentina,"['CAM', 'CM', 'CF']",79,87,Atlanta United,2022 ~ 2025,€39.5M,€10K,2050


In [10]:
def convert_num(y):
    try:
        if y.endswith('K')==True:
            y = list(y)
            y.remove(y[''.join(y).find('K')])
            if ''.join(y).startswith('€')==True:
                y.remove(y[''.join(y).find('€')])
            else:
                pass
            try :
                return int(''.join(y))*1000
            except:
                return y
        elif y.endswith('M')==True:
            y = list(y)
            y.remove(y[''.join(y).find('M')])
            if ''.join(y).startswith('€')==True:
                y = list(y)
                y.remove(y[''.join(y).find('€')])
            else:
                pass
            try :
                return float(''.join(y))*1000000
            except:
                return y
    except:
        return y

In [11]:
convert_num('€230M')

230000000.0

In [12]:
df['Wage'] = df['Wage'].apply(lambda x:convert_num(x))

In [13]:
df.head()

,Player_name,Images,Age,National_team,Positions,Overall,Potential_overall,Current_club,Current_contract,Value,Wage,Total_stats
0,T. Almada,https://cdn.sofifa.net/players/245/371/24_60.png,22,Argentina,"['CAM', 'CM', 'CF']",79,87,Atlanta United,2022 ~ 2025,€39.5M,10000.0,2050
1,L. Palma,https://cdn.sofifa.net/player_0.svg,23,Honduras,['LW'],69,75,Celtic,2023 ~ 2028,€2.2M,22000.0,1794
2,R. Lavia,https://cdn.sofifa.net/players/263/620/24_60.png,19,Belgium,['CDM'],73,86,Chelsea,2023 ~ 2030,€7M,32000.0,1829
3,W. Zaïre-Emery,https://cdn.sofifa.net/players/270/673/24_60.png,17,France,"['CM', 'CDM']",77,89,Paris Saint Germain,2022 ~ 2025,€24M,9000.0,2080
4,Gabri Veiga,https://cdn.sofifa.net/players/258/729/24_60.png,21,Spain,"['CM', 'CAM']",78,89,Al Ahli Jeddah,2023 ~ 2026,€31.5M,28000.0,1944


In [14]:
df['Value'] = df['Value'].apply(lambda x:convert_num(x))

In [15]:
df.head()

,Player_name,Images,Age,National_team,Positions,Overall,Potential_overall,Current_club,Current_contract,Value,Wage,Total_stats
0,T. Almada,https://cdn.sofifa.net/players/245/371/24_60.png,22,Argentina,"['CAM', 'CM', 'CF']",79,87,Atlanta United,2022 ~ 2025,39500000.0,10000.0,2050
1,L. Palma,https://cdn.sofifa.net/player_0.svg,23,Honduras,['LW'],69,75,Celtic,2023 ~ 2028,2200000.0,22000.0,1794
2,R. Lavia,https://cdn.sofifa.net/players/263/620/24_60.png,19,Belgium,['CDM'],73,86,Chelsea,2023 ~ 2030,7000000.0,32000.0,1829
3,W. Zaïre-Emery,https://cdn.sofifa.net/players/270/673/24_60.png,17,France,"['CM', 'CDM']",77,89,Paris Saint Germain,2022 ~ 2025,24000000.0,9000.0,2080
4,Gabri Veiga,https://cdn.sofifa.net/players/258/729/24_60.png,21,Spain,"['CM', 'CAM']",78,89,Al Ahli Jeddah,2023 ~ 2026,31500000.0,28000.0,1944


In [16]:
df['Current_contract_upto'] = df['Current_contract'].apply(lambda x:x.split('~')[1] if len(x.split('~'))>1 else '2050')
df['Current_contract_from'] = df['Current_contract'].apply(lambda x:x.split('~')[0])

In [17]:
df = df.drop('Current_contract', axis = 1)
df.head()

,Player_name,Images,Age,National_team,Positions,Overall,Potential_overall,Current_club,Value,Wage,Total_stats,Current_contract_upto,Current_contract_from
0,T. Almada,https://cdn.sofifa.net/players/245/371/24_60.png,22,Argentina,"['CAM', 'CM', 'CF']",79,87,Atlanta United,39500000.0,10000.0,2050,2025,2022
1,L. Palma,https://cdn.sofifa.net/player_0.svg,23,Honduras,['LW'],69,75,Celtic,2200000.0,22000.0,1794,2028,2023
2,R. Lavia,https://cdn.sofifa.net/players/263/620/24_60.png,19,Belgium,['CDM'],73,86,Chelsea,7000000.0,32000.0,1829,2030,2023
3,W. Zaïre-Emery,https://cdn.sofifa.net/players/270/673/24_60.png,17,France,"['CM', 'CDM']",77,89,Paris Saint Germain,24000000.0,9000.0,2080,2025,2022
4,Gabri Veiga,https://cdn.sofifa.net/players/258/729/24_60.png,21,Spain,"['CM', 'CAM']",78,89,Al Ahli Jeddah,31500000.0,28000.0,1944,2026,2023


In [18]:
df['Positions'] = df['Positions'].apply(lambda x:eval(x))

In [19]:
df = df.explode('Positions')

In [20]:
df.head(5)

,Player_name,Images,Age,National_team,Positions,Overall,Potential_overall,Current_club,Value,Wage,Total_stats,Current_contract_upto,Current_contract_from
0,T. Almada,https://cdn.sofifa.net/players/245/371/24_60.png,22,Argentina,CAM,79,87,Atlanta United,39500000.0,10000.0,2050,2025,2022
0,T. Almada,https://cdn.sofifa.net/players/245/371/24_60.png,22,Argentina,CM,79,87,Atlanta United,39500000.0,10000.0,2050,2025,2022
0,T. Almada,https://cdn.sofifa.net/players/245/371/24_60.png,22,Argentina,CF,79,87,Atlanta United,39500000.0,10000.0,2050,2025,2022
1,L. Palma,https://cdn.sofifa.net/player_0.svg,23,Honduras,LW,69,75,Celtic,2200000.0,22000.0,1794,2028,2023
2,R. Lavia,https://cdn.sofifa.net/players/263/620/24_60.png,19,Belgium,CDM,73,86,Chelsea,7000000.0,32000.0,1829,2030,2023


In [21]:
df = pyd.concat([df,pyd.get_dummies(df['Positions'])], axis = 1)

In [22]:
df.drop(['CAM','CDM','CF','CM','LW']).columns

Index(['Player_name', 'Images', 'Age', 'National_team', 'Positions', 'Overall',
       'Potential_overall', 'Current_club', 'Value', 'Wage', 'Total_stats',
       'Current_contract_upto', 'Current_contract_from', 'CAM', 'CB', 'CDM',
       'CF', 'CM', 'GK', 'LB', 'LM', 'LW', 'LWB', 'RB', 'RM', 'RW', 'RWB',
       'ST'],
      dtype='object')

In [23]:
df

23/12/24 20:05:50 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


,Player_name,Images,Age,National_team,Positions,Overall,Potential_overall,Current_club,Value,Wage,Total_stats,Current_contract_upto,Current_contract_from,CAM,CB,CDM,CF,CM,GK,LB,LM,LW,LWB,RB,RM,RW,RWB,ST
0,T. Almada,https://cdn.sofifa.net/players/245/371/24_60.png,22,Argentina,CAM,79,87,Atlanta United,39500000.0,10000.0,2050,2025,2022,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,T. Almada,https://cdn.sofifa.net/players/245/371/24_60.png,22,Argentina,CM,79,87,Atlanta United,39500000.0,10000.0,2050,2025,2022,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
0,T. Almada,https://cdn.sofifa.net/players/245/371/24_60.png,22,Argentina,CF,79,87,Atlanta United,39500000.0,10000.0,2050,2025,2022,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
1,L. Palma,https://cdn.sofifa.net/player_0.svg,23,Honduras,LW,69,75,Celtic,2200000.0,22000.0,1794,2028,2023,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
2,R. Lavia,https://cdn.sofifa.net/players/263/620/24_60.png,19,Belgium,CDM,73,86,Chelsea,7000000.0,32000.0,1829,2030,2023,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
3,W. Zaïre-Emery,https://cdn.sofifa.net/players/270/673/24_60.png,17,France,CM,77,89,Paris Saint Germain,24000000.0,9000.0,2080,2025,2022,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
3,W. Zaïre-Emery,https://cdn.sofifa.net/players/270/673/24_60.png,17,France,CDM,77,89,Paris Saint Germain,24000000.0,9000.0,2080,2025,2022,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
4,Gabri Veiga,https://cdn.sofifa.net/players/258/729/24_60.png,21,Spain,CM,78,89,Al Ahli Jeddah,31500000.0,28000.0,1944,2026,2023,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
4,Gabri Veiga,https://cdn.sofifa.net/players/258/729/24_60.png,21,Spain,CAM,78,89,Al Ahli Jeddah,31500000.0,28000.0,1944,2026,2023,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,J. Bellingham,https://cdn.sofifa.net/players/270/964/24_60.png,17,England,CAM,64,82,Sunderland,1500000.0,1000.0,1714,2028,2023,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [24]:
df = df.drop('Positions', axis = 1)